In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MAXR', 'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-6a', mod, True, loss=lossfn)[0]

In [10]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
54,RBLX,RROnly,325.74,524.54,850.28,25.498974,0.980000,40.34,42.509998,0.05,9.400000
32,LYFT,RROnly,338.78,423.98,762.77,9.007044,0.952128,9.58,12.870000,0.34,9.380476
62,TOST,RROnly,85.69,95.23,180.92,13.317792,0.958904,21.07,25.150000,0.19,9.372747
52,PTON,RROnly,694.80,756.02,1450.82,9.274025,0.947977,8.37,12.020000,0.44,9.361534
73,ZS,RROnly,1955.38,1536.80,3492.19,43.678714,0.957447,142.39,127.089996,-0.11,9.187621
24,GLBE,RROnly,173.82,354.48,528.30,18.462670,0.945055,34.73,39.619999,0.14,9.058085
29,IOT,RROnly,52.82,63.80,116.62,9.116216,0.934426,24.31,20.680000,-0.15,8.907104
48,PD,RROnly,294.02,319.09,613.11,13.730500,0.914894,23.00,30.639999,0.33,8.891038
20,ETSY,RROnly,1310.38,1639.48,2949.85,35.351124,0.920213,85.01,69.589996,-0.18,8.758865
34,MBLY,RROnly,37.58,37.86,75.44,26.427030,0.944444,43.56,40.209999,-0.08,8.750000


In [ ]:
findata.EPOCHS=200
later = [ 'SMH']
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX', 
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM', 
            'ONLN', 'QQQ', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-5a1', mod, True)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
5,DTEC,RROnly,190.76,162.67,353.43,23.080429,0.968085,36.05,34.439999,-0.04,8.882979
22,SMOG,RROnly,755.76,684.98,1440.74,51.844643,0.952128,115.62,109.160004,-0.06,8.723404
14,IWM,RROnly,875.32,668.12,1543.44,97.725729,0.962766,174.45,166.809998,-0.04,8.696809
0,ARKF,RROnly,229.73,222.12,451.85,14.213349,0.941489,18.52,17.180000,-0.07,8.579027
36,XLP,RROnly,229.13,124.50,353.63,45.573143,0.936170,77.18,71.949997,-0.07,8.449848
39,XLV,RROnly,414.51,282.96,697.47,72.271937,0.930851,133.59,124.800003,-0.07,8.320669
41,XME,RROnly,285.36,196.50,481.86,14.492709,0.946809,48.47,46.759998,-0.04,8.138298
7,FPX,RROnly,454.39,433.80,888.19,55.621895,0.909574,80.80,74.339996,-0.08,7.965426
2,ARKW,RROnly,689.59,770.32,1459.90,38.322203,0.930851,48.84,46.610001,-0.05,7.925532
17,MGK,RROnly,987.58,554.01,1541.59,111.073978,0.946809,208.60,203.309998,-0.03,7.695035


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL', 
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC', 'ASTR', 'ATOM',
            'BBBY', 'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE', 
            'INLX',  'JOBY',  'KIND', 'KULR',
            'LAZR', 'MMMB', 'MTTR', 
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST', 
            'RAD', 'REAL', 'RIDE', 'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'YEXT']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-4a', mod, False)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
39,OUST,RROnly,255.96,687.94,943.90,6.203406,0.915254,4.94,6.68,0.35,5.141718
36,ONDS,RROnly,37.30,55.89,93.19,1.120860,0.900000,1.08,1.68,0.55,5.112299
22,GOVX,RROnly,27.19,38.59,65.78,0.725319,0.900000,0.65,0.86,0.33,4.991087
37,OPEN,RROnly,69.15,176.02,245.17,1.499331,0.858209,1.95,4.70,1.41,4.947727
46,STEM,RROnly,130.69,263.51,394.20,4.841042,0.906780,4.45,5.36,0.20,4.867896
40,RAD,RROnly,154.26,144.57,298.83,2.510473,0.872340,1.86,2.57,0.38,4.795468
11,BSGM,RROnly,44.65,59.65,104.30,0.513760,0.845745,1.32,2.24,0.70,4.754604
32,MTTR,RROnly,59.95,71.30,131.25,2.772340,0.872549,2.77,3.61,0.30,4.707805
18,EVGO,RROnly,61.52,94.53,156.05,4.170255,0.892857,5.72,6.76,0.18,4.656863
33,NEPH,RROnly,43.88,64.93,108.81,1.105322,0.856383,1.42,1.92,0.36,4.638611


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-5a', mod4, True)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
30,PYPL,RROnly,1353.03,1379.66,2732.69,65.542773,0.946809,61.69,69.279999,0.12,6.319656
16,DIS,RROnly,799.46,684.83,1484.30,86.234207,0.920213,91.99,101.940002,0.11,5.847610
20,INTC,RROnly,285.19,232.51,517.70,25.150653,0.930851,28.95,30.950001,0.07,5.661094
26,NKE,RROnly,635.23,644.28,1279.51,62.991202,0.920213,120.21,127.059998,0.06,5.243161
3,AMD,RROnly,741.50,552.34,1293.84,28.368724,0.882979,95.26,103.360001,0.09,5.006754
10,CDNS,RROnly,1060.58,446.66,1507.24,56.824049,0.909574,201.16,188.479996,-0.06,4.989868
23,META,RROnly,1610.57,1315.62,2926.19,94.101420,0.920213,233.81,246.380005,0.05,4.977204
1,ACN,RROnly,1326.07,1060.46,2386.53,141.529015,0.920213,277.19,261.959991,-0.05,4.977204
24,MMC,RROnly,629.27,439.46,1068.73,74.663228,0.904255,181.22,169.639999,-0.06,4.863222
5,AVGO,RROnly,2632.96,1319.35,3952.31,157.490616,0.872340,631.15,575.739990,-0.09,4.812563


In [ ]:
findata.EPOCHS=200
tickers5 = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA', 
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-4a', mod5, False)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
35,MGA,RROnly,358.84,368.67,727.51,23.588843,0.925532,52.73,45.650002,-0.13,5.465279
58,STLA,RROnly,98.60,60.94,159.54,4.824281,0.941489,16.78,15.480000,-0.08,5.262158
45,ONON,RROnly,79.41,117.06,196.47,16.910025,0.875000,26.69,33.020000,0.24,5.208333
49,PROSF,RROnly,564.74,489.57,1054.30,43.945877,0.929825,73.00,66.389999,-0.09,5.082150
66,VNT,RROnly,89.11,102.40,191.51,17.048118,0.909091,29.29,25.820000,-0.12,4.978355
27,GLOB,RROnly,1676.77,1420.68,3097.45,78.214478,0.872340,167.40,133.630005,-0.20,4.954407
39,LVMUY,RROnly,964.65,397.13,1361.78,61.376531,0.893617,189.47,162.160004,-0.14,4.863222
20,ENGGY,RROnly,41.30,27.93,69.23,6.780015,0.920213,9.56,8.740000,-0.09,4.799899
8,BASFY,RROnly,60.51,71.44,131.95,8.438459,0.893617,12.89,11.240000,-0.13,4.746318
65,VALE,RROnly,84.30,79.44,163.74,5.234479,0.914894,13.86,12.620000,-0.09,4.643701


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-5b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
16,KEMQ,RROnly,120.41,130.32,250.73,11.834671,0.930851,14.44,15.360000,0.06,8.156028
9,EWM,RROnly,70.16,91.57,161.73,17.312569,0.941489,21.57,22.350000,0.04,7.952128
5,ECH,RROnly,150.36,154.91,305.27,16.031340,0.914894,29.92,27.680000,-0.07,7.933131
2,CUBS,RROnly,16.63,42.69,59.32,18.684955,0.917647,19.35,20.430000,0.06,7.803922
15,ILF,RROnly,143.90,101.28,245.18,12.888522,0.930851,25.43,24.350000,-0.04,7.579787
3,FLN,RROnly,89.70,88.62,178.31,9.137238,0.930851,18.63,17.980000,-0.03,7.003546
1,EWD,RROnly,153.31,139.70,293.00,20.033818,0.909574,36.29,35.330002,-0.03,6.081560
6,EWA,RROnly,86.11,82.28,168.39,11.578962,0.904255,23.00,22.250000,-0.03,5.851064
0,ARGT,RROnly,171.44,141.25,312.69,14.281111,0.872340,40.76,39.250000,-0.04,5.531915
19,XCEM,RROnly,99.46,96.24,195.70,16.238212,0.909574,27.50,26.830000,-0.02,4.574468


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN', 
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB', 
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK', 
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
           
mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-4b', mod7, False)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
74,XOM,RROnly,474.72,319.22,793.94,27.427862,0.962766,104.97,97.239998,-0.07,9.095745
32,HSII,RROnly,174.63,195.38,370.01,17.720352,0.941489,25.10,21.930000,-0.13,8.964812
72,WHR,RROnly,949.27,814.61,1763.88,59.885397,0.930851,134.22,112.110001,-0.16,8.876330
16,CG,RROnly,249.02,178.18,427.20,15.931623,0.936170,27.95,24.440001,-0.13,8.870704
30,GLW,RROnly,145.55,149.66,295.21,16.738191,0.936170,31.53,27.459999,-0.13,8.870704
1,APO,RROnly,402.53,249.40,651.93,22.299497,0.925532,65.42,53.930000,-0.18,8.841608
68,URI,RROnly,2370.86,1771.94,4142.80,76.043612,0.920213,350.97,277.450012,-0.21,8.822188
4,ARE,RROnly,790.82,792.45,1583.27,107.178950,0.925532,111.77,128.699997,0.15,8.758865
26,FCX,RROnly,291.51,200.09,491.60,5.886090,0.914894,34.77,27.590000,-0.21,8.743668
70,VZ,RROnly,133.48,155.76,289.24,34.627340,0.952128,35.00,37.080002,0.06,8.723404


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'JJU', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-5b', mod8, True)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
2,JJU,RROnly,208.96,170.59,379.55,33.160814,0.835106,46.15,48.750000,0.06,5.602837
4,SLV,RROnly,117.73,92.20,209.93,11.551418,0.893617,21.38,20.790001,-0.03,5.390071
1,ISHG,RROnly,76.59,98.72,175.31,63.653912,0.877660,68.58,70.419998,0.03,4.698582
0,GLD,RROnly,513.71,337.50,851.21,126.691556,0.909574,180.92,185.389999,0.02,4.574468
10,WEAT,RROnly,31.62,36.58,68.20,5.029175,0.856383,6.26,6.070000,-0.03,3.776596
5,SOYB,RROnly,70.55,40.09,110.64,13.637010,0.819149,24.92,23.990000,-0.04,3.670213
7,TLT,RROnly,263.77,364.65,628.42,92.102580,0.909574,101.09,102.570000,0.01,0.053191
3,PDBC,RROnly,48.27,30.98,79.25,6.613453,0.898936,13.70,13.560000,-0.01,0.000000
6,TIP,RROnly,100.27,129.75,230.01,96.636619,0.904255,107.73,109.190002,0.01,0.000000
8,UUP,RROnly,38.26,19.45,57.72,24.016293,0.829787,28.59,28.430000,-0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-4b', mod9, False)[0]

In [8]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
6,MATIC-USD,RROnly,20.49,13.53,34.02,0.094402,0.925532,0.90,1.050000,0.17,6.172895
7,XRP-USD,RROnly,11.79,8.48,20.27,0.261491,0.872340,0.53,0.670000,0.28,5.775076
0,ADA-USD,RROnly,11.26,11.43,22.69,0.140281,0.813830,0.38,0.900000,1.37,5.677179
4,ETH-USD,RROnly,27432.28,23668.69,51100.97,455.574312,0.845745,1907.57,2288.949951,0.20,5.269757
2,DOT-USD,RROnly,356.77,268.19,624.96,5.048052,0.787234,5.29,6.190000,0.17,4.371536
1,BTC-USD,RROnly,430350.27,262804.42,693154.68,11450.542228,0.840426,27291.44,29775.619141,0.09,4.229990
3,DOGE-USD,RROnly,3.37,1.82,5.19,0.014787,0.771277,0.07,0.060000,-0.17,4.163687
5,LTC-USD,RROnly,1538.38,1698.01,3236.40,50.189092,0.813830,95.30,92.949997,-0.02,0.000000


In [ ]:
findata.EPOCHS=200
save = ['CF']
tickers10 = ['AAPL', 'AN', 'AZN', 'BJ', 'BLDR',
             'COST', 'CPRX', 'CROX', 'DHR', 'DLTR', 'DY',
             'ELF', 'EQNR',  'HRMY', 'HUM',  'LNTH',
             'MCK',  'MPWR', 'NVDA', 'ON', 'PWR', 'QQQ',  'SWAV', 
             'TSLA', 'UNH', 'URI', 'VRRM', 'XLE']
mod10 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df10 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod10, False)[0]

loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df10, 0.07, 1)
df10.sort_values('Alloc', ascending=False)

In [ ]:
findata.G_LOOKUP_STEP = 5
mod11 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df11 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod11, False)[0]


loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-5-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df11, 0.07, 1)
df11.sort_values('Alloc', ascending=False)

In [11]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
ipos-6a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,219,173 bytes  received 35 bytes  6,438,416.00 bytes/sec
total size is 138,134,984  speedup is 42.91
